In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-347601-6e0b822fd842.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#date = 0 #当日は0、前日は1・・・
#
date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 18001005 # 店舗ID
SPREADSHEET_KEY = '1IY8cCXjr2WOlXKbe0nkh7s-8SXbyr2oFz7c2BHWFqeI' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 577 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\garden\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p40row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p40table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "40":
                    p40table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "40":
                        p40table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.4RAW'!F" + str(int(p40row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p40table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

The version of chrome cannot be detected. Trying with latest driver version


['08/19', '025474', '1', '31538.0', '-10531.0', '21', '11']
['08/19', '025364', '2', '41282.0', '7223.0', '58', '15']
['08/19', '025758', '3', '40000.0', '-19860.0', '13', '7']
['08/19', '025579', '4', '53333.0', '-20161.0', '13', '4']
['08/19', '025826', '5', '31026.0', '-3309.0', '15', '5']
['08/19', '025404', '6', '39487.0', '603.0', '43', '19']
['08/19', '025737', '7', '43333.0', '-3610.0', '59', '6']
['08/19', '025807', '8', '34359.0', '18357.0', '59', '8']
['08/19', '025287', '9', '41026.0', '-12638.0', '47', '9']
['08/19', '025317', '10', '32308.0', '-4814.0', '28', '14']
['08/19', '025110', '11', '26410.0', '-1203.0', '52', '13']
['08/19', '025489', '12', '46154.0', '-18356.0', '7', '5']
['08/19', '025849', '13', '41538.0', '2709.0', '24', '6']
['08/19', '025754', '14', '28718.0', '7223.0', '49', '8']
['08/19', '025667', '15', '33846.0', '16853.0', '27', '3']
['08/19', '025469', '16', '47436.0', '-8124.0', '21', '5']
['08/19', '025826', '17', '32564.0', '-9930.0', '11', '4']
['

['08/19', '025807', '140', '19231.0', '4515.0', '26', '6']
['08/19', '025480', '141', '33590.0', '-13240.0', '39', '14']
['08/19', '025864', '142', '16667.0', '9631.0', '29', '3']
['08/19', '025559', '143', '19231.0', '1506.0', '23', '8']
['08/19', '025473', '144', '31795.0', '-2106.0', '22', '7']
['08/19', '025718', '145', '21282.0', '603.0', '85', '10']
['08/19', '025357', '146', '14615.0', '2107.0', '10', '1']
['08/19', '025554', '147', '27179.0', '12038.0', '42', '6']
['08/19', '025455', '148', '11026.0', '904.0', '9', '3']
['08/19', '025225', '149', '16410.0', '603.0', '18', '16']
['08/19', '025501', '150', '21026.0', '13543.0', '47', '5']
['08/19', '025344', '151', '23333.0', '-3309.0', '18', '6']
['08/19', '025852', '152', '19744.0', '-1203.0', '23', '3']
['08/19', '025543', '153', '16667.0', '-1203.0', '10', '4']
['08/19', '025264', '154', '21795.0', '-902.0', '14', '2']
['08/19', '025285', '155', '22308.0', '-7522.0', '10', '6']
['08/19', '025466', '156', '21026.0', '-1805.0',

['08/19', '025553', '278', '38974.0', '4816.0', '30', '9']
['08/19', '025553', '279', '36154.0', '-2707.0', '24', '8']
['08/19', '025521', '280', '33846.0', '1205.0', '36', '14']
['08/19', '025521', '281', '41282.0', '8728.0', '51', '24']
['08/19', '025521', '282', '21282.0', '904.0', '24', '8']
['08/19', '025521', '283', '35385.0', '-6018.0', '25', '10']
['08/19', '025605', '284', '25897.0', '-2106.0', '24', '11']
['08/19', '025605', '285', '10513.0', '-3008.0', '7', '4']
['08/19', '025605', '286', '18462.0', '603.0', '17', '6']
['08/19', '025655', '287', '16154.0', '9330.0', '64', '9']
['08/19', '025655', '288', '36923.0', '3311.0', '88', '23']
['08/19', '025521', '289', '13077.0', '-1203.0', '13', '5']
['08/19', '025521', '290', '9231.0', '-2406.0', '5', '2']
['08/19', '025605', '291', '5897.0', '1506.0', '8', '4']
['08/19', '025605', '292', '7692.0', '-902.0', '6', '5']
['08/19', '025863', '293', '10513.0', '1205.0', '18', '9']
['08/19', '025877', '294', '48718.0', '-6018.0', '44',

['08/19', '025867', '418', '5385.0', '-4513.0', '0', '0']
['08/19', '025867', '419', '8462.0', '-6619.0', '0', '0']
['08/19', '025867', '420', '10000.0', '-2406.0', '4', '3']
['08/19', '025850', '421', '7436.0', '-5115.0', '1', '1']
['08/19', '025850', '422', '5641.0', '8728.0', '6', '2']
['08/19', '025850', '423', '2564.0', '-1203.0', '1', '1']
['08/19', '025734', '424', '23333.0', '16853.0', '23', '1']
['08/19', '025818', '425', '11538.0', '4515.0', '10', '3']
['08/19', '025818', '426', '16410.0', '3311.0', '11', '3']
['08/19', '025818', '427', '17692.0', '3913.0', '13', '3']
['08/19', '025852', '428', '2821.0', '4816.0', '15', '1']
['08/19', '025879', '429', '11026.0', '1205.0', '10', '2']
['08/19', '025879', '430', '2821.0', '-902.0', '2', '1']
['08/19', '025879', '431', '5897.0', '-3911.0', '1', '1']
['08/19', '025879', '432', '15897.0', '1506.0', '17', '7']
['08/19', '025837', '433', '14103.0', '-9930.0', '2', '2']
['08/19', '025837', '434', '18974.0', '-14444.0', '1', '1']
['08/

['08/19', '025869', '558', '0', '0', '0', '0']
['08/19', '025895', '559', '46154.0', '-12638.0', '17', '4']
['08/19', '025895', '560', '49231.0', '-25578.0', '12', '5']
['08/19', '025895', '561', '45641.0', '-21064.0', '12', '3']
['08/19', '025895', '562', '52051.0', '25279.0', '49', '8']
['08/19', '025895', '563', '45897.0', '35510.0', '51', '10']
['08/19', '025895', '564', '40769.0', '9931.0', '30', '6']
['08/19', '025895', '565', '40769.0', '-10531.0', '20', '8']
['08/19', '025895', '566', '43846.0', '-16550.0', '14', '4']
['08/19', '025895', '567', '47179.0', '8126.0', '35', '9']
['08/19', '025606', '568', '14103.0', '-2707.0', '6', '1']
['08/19', '025606', '569', '4872.0', '-1203.0', '2', '1']
['08/19', '025606', '570', '10000.0', '-8124.0', '0', '0']
['08/19', '025764', '571', '8974.0', '-3610.0', '2', '1']
['08/19', '025764', '572', '7949.0', '3010.0', '6', '1']
['08/19', '025856', '573', '14872.0', '2709.0', '10', '4']
['08/19', '025856', '574', '30256.0', '-13842.0', '7', '4']

['08/19', '120010', '693', '2341', '-1250.0', '4', '4', '--']
['08/19', '120010', '694', '3968', '-1204.0', '12', '8', '--']
['08/19', '120010', '695', '5953', '139.0', '24', '17', '--']
['08/19', '120010', '696', '2864', '370.0', '12', '7', '--']
['08/19', '120010', '697', '4553', '-324.0', '15', '13', '--']
['08/19', '120010', '698', '4899', '-926.0', '15', '12', '--']
['08/19', '120010', '699', '1899', '-972.0', '4', '3', '--']
['08/19', '120010', '700', '3808', '-509.0', '14', '8', '--']
['08/19', '120010', '701', '3582', '278.0', '17', '4', '--']
['08/19', '120010', '702', '3666', '-1019.0', '8', '12', '--']
['08/19', '120122', '703', '6498', '-2454.0', '48', '18', '--']
['08/19', '120122', '704', '2302', '-1667.0', '12', '5', '--']
['08/19', '120122', '705', '5150', '-3657.0', '29', '14', '--']
['08/19', '120122', '706', '4985', '-139.0', '55', '13', '--']
['08/19', '120122', '707', '1162', '-417.0', '10', '2', '--']
['08/19', '120122', '708', '5529', '463.0', '63', '13', '--']
[

['08/19', '120076', '828', '548', '278.0', '0', '0', '4']
['08/19', '120076', '829', '4503', '185.0', '0', '0', '21']
['08/19', '120094', '830', '11', '0', '0', '0', '--']
['08/19', '120094', '831', '11', '0', '0', '0', '--']
['08/19', '120080', '832', '2122', '-509.0', '14', '0', '18']
['08/19', '120080', '833', '267', '-185.0', '1', '0', '1']
['08/19', '120080', '834', '877', '-1065.0', '4', '0', '1']
['08/19', '120126', '835', '3381', '-2685.0', '0', '12', '6']
['08/19', '120126', '836', '7008', '2500.0', '0', '19', '51']
['08/19', '120126', '837', '664', '-880.0', '0', '1', '--']
['08/19', '120126', '838', '7480', '463.0', '0', '13', '46']
['08/19', '120126', '839', '6091', '4491.0', '0', '13', '44']
['08/19', '120073', '840', '5002', '648.0', '0', '12', '40']
['08/19', '120073', '841', '5284', '139.0', '0', '14', '37']
['08/19', '120073', '842', '1143', '-648.0', '0', '3', '5']
['08/19', '120073', '843', '8083', '5000.0', '0', '24', '80']
['08/19', '120073', '844', '3425', '-1389.

{'spreadsheetId': '1IY8cCXjr2WOlXKbe0nkh7s-8SXbyr2oFz7c2BHWFqeI',
 'updates': {'spreadsheetId': '1IY8cCXjr2WOlXKbe0nkh7s-8SXbyr2oFz7c2BHWFqeI',
  'updatedRange': 'S5RAW!F2186:M2224',
  'updatedRows': 39,
  'updatedColumns': 8,
  'updatedCells': 312}}

In [12]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p40table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "40":
                    p40table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "40":
                        p40table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.4RAW'!F" + str(int(p40row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p40table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/08', '025474', '1', '7949.0', '904.0', '10', '2']
['08/08', '025364', '2', '16410.0', '-5416.0', '10', '5']
['08/08', '025758', '3', '2564.0', '-2106.0', '0', '0']
['08/08', '025579', '4', '10256.0', '-3610.0', '3', '1']
['08/08', '025826', '5', '1538.0', '-1203.0', '0', '0']
['08/08', '025404', '6', '17179.0', '-4814.0', '12', '3']
['08/08', '025737', '7', '5641.0', '-3309.0', '3', '1']
['08/08', '025807', '8', '15385.0', '11135.0', '29', '7']
['08/08', '025287', '9', '10769.0', '904.0', '26', '3']
['08/08', '025317', '10', '5385.0', '-2106.0', '3', '2']
['08/08', '025110', '11', '12051.0', '2709.0', '31', '5']
['08/08', '025489', '12', '4359.0', '-3309.0', '0', '0']
['08/08', '025849', '13', '22821.0', '-5115.0', '9', '3']
['08/08', '025754', '14', '8974.0', '-4513.0', '4', '1']
['08/08', '025667', '15', '12564.0', '-7823.0', '2', '2']
['08/08', '025469', '16', '16154.0', '-11434.0', '1', '1']
['08/08', '025826', '17', '6923.0', '4816.0', '7', '1']
['08/08', '025800', '18', '564

['08/08', '025718', '145', '16667.0', '-902.0', '58', '7']
['08/08', '025357', '146', '0', '0', '0', '0']
['08/08', '025554', '147', '769.0', '-902.0', '0', '0']
['08/08', '025455', '148', '0', '0', '0', '0']
['08/08', '025225', '149', '0', '0', '0', '0']
['08/08', '025501', '150', '8974.0', '5418.0', '21', '4']
['08/08', '025344', '151', '3590.0', '-2406.0', '0', '0']
['08/08', '025852', '152', '12308.0', '10232.0', '40', '2']
['08/08', '025543', '153', '513.0', '-902.0', '0', '0']
['08/08', '025264', '154', '4103.0', '-2707.0', '0', '0']
['08/08', '025285', '155', '11538.0', '-1504.0', '8', '3']
['08/08', '025466', '156', '513.0', '-902.0', '0', '0']
['08/08', '025814', '157', '4615.0', '-3911.0', '0', '0']
['08/08', '025293', '158', '8718.0', '-902.0', '7', '3']
['08/08', '025357', '159', '4872.0', '-3309.0', '0', '0']
['08/08', '025766', '160', '8205.0', '13242.0', '23', '2']
['08/08', '025849', '161', '15385.0', '11737.0', '16', '4']
['08/08', '025825', '162', '10513.0', '-5717.0'

['08/08', '025521', '289', '7692.0', '2408.0', '11', '3']
['08/08', '025521', '290', '0', '0', '0', '0']
['08/08', '025605', '291', '0', '0', '0', '0']
['08/08', '025605', '292', '256.0', '-902.0', '0', '0']
['08/08', '025863', '293', '0', '0', '0', '0']
['08/08', '025877', '294', '11026.0', '-4212.0', '5', '3']
['08/08', '025877', '295', '513.0', '904.0', '3', '2']
['08/08', '025877', '296', '5128.0', '3010.0', '11', '4']
['08/08', '025877', '297', '769.0', '904.0', '2', '1']
['08/08', '025489', '298', '0', '0', '0', '0']
['08/08', '025489', '299', '769.0', '-902.0', '0', '0']
['08/08', '025489', '300', '3590.0', '-3008.0', '0', '0']
['08/08', '025489', '301', '7692.0', '-3911.0', '1', '1']
['08/08', '025579', '302', '3333.0', '-2406.0', '0', '0']
['08/08', '025579', '303', '3077.0', '-2406.0', '0', '0']
['08/08', '025579', '304', '3077.0', '-2406.0', '0', '0']
['08/08', '025579', '305', '0', '0', '0', '0']
['08/08', '025579', '306', '3846.0', '-3008.0', '0', '0']
['08/08', '025865', 

['08/08', '025801', '436', '3333.0', '27686.0', '20', '1']
['08/08', '025849', '437', '0', '0', '0', '0']
['08/08', '025849', '438', '4103.0', '-3610.0', '0', '0']
['08/08', '025355', '439', '513.0', '-902.0', '0', '0']
['08/08', '025825', '440', '0', '0', '0', '0']
['08/08', '025825', '441', '0', '0', '0', '0']
['08/08', '025825', '442', '0', '0', '0', '0']
['08/08', '025665', '443', '19487.0', '6019.0', '15', '3']
['08/08', '025665', '444', '12564.0', '302.0', '7', '3']
['08/08', '025665', '445', '12821.0', '-9629.0', '1', '1']
['08/08', '025665', '446', '19744.0', '-3911.0', '10', '4']
['08/08', '025665', '447', '13333.0', '21367.0', '20', '1']
['08/08', '025665', '448', '18462.0', '-11133.0', '3', '1']
['08/08', '025665', '449', '14615.0', '7825.0', '14', '3']
['08/08', '025665', '450', '21282.0', '-11434.0', '5', '2']
['08/08', '025677', '451', '11026.0', '5418.0', '10', '3']
['08/08', '025677', '452', '10256.0', '-1203.0', '6', '1']
['08/08', '025677', '453', '14359.0', '-1504.0'

['08/08', '120128', '584', '1417', '787.0', '0', '1', '6']
['08/08', '120142', '585', '1826', '-93.0', '0', '4', '17']
['08/08', '120142', '586', '1532', '3611.0', '0', '4', '24']
['08/08', '120142', '587', '2426', '231.0', '0', '6', '19']
['08/08', '120142', '588', '4443', '-2593.0', '0', '8', '28']
['08/08', '120140', '589', '3557', '-1620.0', '0', '1', '34']
['08/08', '120140', '590', '1845', '139.0', '0', '0', '33']
['08/08', '120140', '591', '634', '-556.0', '0', '0', '11']
['08/08', '120140', '592', '3436', '-4120.0', '0', '0', '44']
['08/08', '120140', '593', '881', '-463.0', '0', '0', '22']
['08/08', '120140', '594', '1587', '-1157.0', '0', '0', '22']
['08/08', '120110', '595', '1370', '278.0', '11', '1', '--']
['08/08', '120110', '596', '1444', '602.0', '11', '5', '--']
['08/08', '120110', '597', '1346', '231.0', '8', '8', '--']
['08/08', '120110', '598', '2778', '-3102.0', '5', '3', '--']
['08/08', '120110', '599', '2405', '-972.0', '9', '10', '--']
['08/08', '120110', '600',

['08/08', '120122', '720', '3676', '-556.0', '37', '12', '--']
['08/08', '120121', '721', '602', '-370.0', '0', '1', '7']
['08/08', '120121', '722', '2188', '972.0', '0', '5', '34']
['08/08', '120121', '723', '201', '-278.0', '0', '0', '--']
['08/08', '120121', '724', '141', '-185.0', '0', '0', '--']
['08/08', '120123', '725', '505', '-231.0', '0', '1', '3']
['08/08', '120123', '726', '0', '0', '0', '0', '--']
['08/08', '120034', '727', '88', '-93.0', '0', '0', '--']
['08/08', '120034', '728', '2177', '-1435.0', '0', '2', '11']
['08/08', '120129', '729', '1191', '972.0', '0', '4', '12']
['08/08', '120129', '730', '2417', '-1528.0', '0', '7', '9']
['08/08', '120129', '731', '1713', '-972.0', '0', '6', '8']
['08/08', '120129', '732', '1756', '-185.0', '0', '8', '9']
['08/08', '120145', '733', '3304', '-2315.0', '0', '5', '15']
['08/08', '120145', '734', '4047', '-139.0', '0', '7', '41']
['08/08', '120145', '735', '2832', '-324.0', '0', '4', '26']
['08/08', '120145', '736', '2832', '-694.

['08/08', '119960', '858', '48', '0', '0', '1', '--']
['08/08', '119991', '859', '1151', '278.0', '11', '0', '3']
['08/08', '120099', '860', '975', '-324.0', '2', '0', '--']
['08/08', '120014', '861', '0', '0', '0', '0', '--']
['08/08', '120137', '862', '3466', '139.0', '9', '10', '19']
['08/08', '120016', '863', '608', '417.0', '0', '0', '6']
['08/08', '120072', '864', '0', '0', '0', '0', '--']
['08/08', '120089', '865', '35', '0', '0', '0', '--']
['08/08', '120015', '866', '239', '-324.0', '0', '1', '--']
['08/08', '120094', '867', '827', '509.0', '0', '2', '20']
['08/08', '119995', '868', '0', '0', '0', '0', '--']
['08/08', '120041', '869', '1025', '-1157.0', '0', '3', '--']
['08/08', '119950', '870', '1331', '-1389.0', '0', '1', '1']
['08/08', '120108', '871', '4425', '-1157.0', '0', '0', '21']
['08/08', '120034', '872', '2503', '-2037.0', '0', '2', '11']
['08/08', '119897', '873', '2276', '185.0', '20', '0', '15']
['08/08', '119940', '874', '1807', '-648.0', '7', '0', '5']
['08/08

{'spreadsheetId': '1IY8cCXjr2WOlXKbe0nkh7s-8SXbyr2oFz7c2BHWFqeI',
 'updates': {'spreadsheetId': '1IY8cCXjr2WOlXKbe0nkh7s-8SXbyr2oFz7c2BHWFqeI',
  'updatedRange': 'S5RAW!F1757:M1795',
  'updatedRows': 39,
  'updatedColumns': 8,
  'updatedCells': 312}}